In [29]:
from openai import OpenAI
from secret import OPENAI_API_KEY
import base64
import os
import json
import re

In [30]:
client = OpenAI(api_key=OPENAI_API_KEY)
model = "gpt-4o-mini"

In [31]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [39]:
def send_chat_request(message, extracting_json=False):
    try:
        response = client.chat.completions.create(
            model=model, messages=message
        )

        result = response.choices[0].message.content.strip()
        if extracting_json:
            result = extract_json(result)

        return result

    except Exception as e:
        print(f"Erreur OpenAI : {e}")

In [41]:
def chat_get_key_words(base64_images):

    # Liste pour chaque image et chaque texte associé
    content_list = []
    for i, base64_image in enumerate(base64_images):
        content_list.append({
            "type": "text",
            "text": f"Décris moi toutes les images avec 5 mots-clés. Le format est le suivant Image i : [mot-clé1, mot-clé2, mot-clé3, mot-clé4, mot-clé5]"
        })
        content_list.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
        })

    messages = [
        {
            "role": "user",
            "content": content_list
        }
    ]

    return send_chat_request(messages, extracting_json=False)


In [34]:
def extract_json(response_text):
    """
    Extrait la portion JSON (délimitée par {}) de la réponse textuelle pour seulement avoir le dictionnaire et non le texte généré par l'ia.
    """
    match = re.search(r'\{.*\}', response_text, re.DOTALL)
    if match:
        json_str = match.group()
        try:
            return json.loads(json_str)
        except Exception as e:
            print(f"Erreur lors du chargement du JSON : {e}")
            return None
    else:
        print("Aucun JSON trouvé dans la réponse.")
        return None

In [44]:
def chat_get_categories(keywords_output, image_paths):
    """
    Utilise les mots-clés extraits pour regrouper les images similaires en catégories.
    Les images sont identifiées par leur ordre dans la liste.
    """

    # Préparation d'un prompt détaillé incluant le résultat des mots-clés et l'ordre des images
    prompt = f"""Voici les listes de mots-clés obtenues pour chaque image (dans l'ordre) : {keywords_output}

    L'ordre des images est le suivant : {chr(10).join([f"Image {i} : {path}" for i, path in enumerate(image_paths)])}

    En te basant sur ces informations, regroupe les images similaires dans des catégories. Une catégorie est décrite par un seul mot-clé.
    Retourne le résultat au format JSON en indiquant pour chaque catégorie la liste des indices des images (par exemple : "Image 1", "Image 3", etc.) qui appartiennent à cette catégorie.
    Le format attendu est :
    {{
        "categorie1": [ "Image 1", "Image 3" ],
        "categorie2": [ "Image 2", "Image 4" ],
        ...
    }}
    """
    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]

    return send_chat_request(messages, extracting_json=True)

In [46]:
directory = "test_data"
allowed_extensions = {".jpg", ".jpeg", ".png"}
image_paths = [
    os.path.join(directory, filename)
    for filename in os.listdir(directory)
    if os.path.splitext(filename)[1].lower() in allowed_extensions
]
base64_images = [encode_image(path) for path in image_paths]

In [47]:
keywords_output = chat_get_key_words(base64_images)
print("Mots-clés par image :")
print(keywords_output)

Mots-clés par image :
Image 1 : [lasagnes, plat, table, nourriture, ustensiles]


9s -> 1 image

22s - 42s -> 4 images (pas constant)

In [45]:
categories = chat_get_categories(keywords_output, image_paths)
print("Catégorisation des images :")
print(categories)

Catégorisation des images :
{'travail': ['Image 1'], 'nature': ['Image 2'], 'cuisine': ['Image 3'], 'magasin': ['Image 4']}
